In [ ]:
# Scratch workbook for developing operational local join count statistics


# Reviewing Anselin and Li 2019

Univariate case: a local form of the so-called BB join count (i.e. where observations with a value of 1 occur in spatially adjacent locations, corresponding to positive spatial autocorrelation)

Approach is appropriate 'in a data conext where all potential locations are observed (i.e. lattice data structure)'

'For binary variables coded as 0 and 1, the global spatial autocorrelation statistic of choice is the join count statistic'

Types of binary joins
Black = 1
White = 0

- Black-black (BB): 1-1
- White-White (WW): 0-0
- Black-White (BW): 0-1 (why is black first here if it is 0-1 and not 1-0? cannot tell if it is word or number driven in naming of pairs. a strange wording oddity!)

Focus is on identifying co-occurrences of uncommon events, we focus on BB join counts

## Global BB join count statistic

$$ BB = \sum_{i} \sum_j w_{ij} x_{i} x_{j}$$

where $w_ij$ are the elements of a binary spatial weights matrix that specifies whether locations i and j are adjoining


## The local BB statistic

$$ BB_i = x_i \sum_{j} w_ij x_j $$

A count of the the neighbors with an observations of x_j = 1 for those locations where x_i = 1. For all locations with x_i = 0, the statistic is zero. Written out in lay terms:

$$ BB_i = 1 * \sum_j w_{ij} (x_j==1) $$

## Why this statistic? 

'...assess whether location siwth an event (i.e., $x_i = 1$) are surrounded by more locations with events than would be the case under spatial randomness



## Inference (Anselin and Li 2019 Section 2.3)

How likely a given pattern is to occur under a null hypothesis of spatial randomness. Two approaches:

1. Conditional probability of a configuration observed around a given location (i.e., x = 1) using the properties of a hypergeometric distribution

'Given a total number of events in the sample of $N$ observations as $P$, we consider the number of neighbors of location $i$ for which $x_i$ = 1 (i.e. conditional upon observing 1 at this location). the number of neighbors with $x_j = 1$ is represented by $p_i$.'

The propobability of observing exactly $p_i=p$ conditional upon $x_i=1$ follows the hypergeometric distribution for $N-1$ datapoints and $P-1$ events.

$$ Prob [p_{i} = p | x_{i} = 1] = \frac{\binom{P-1}{p} \binom{N-P-2}{k_i-p}}{\binom{N-1}{k_i}} $$

where $k_i$ is the number of neighbors of observation $i$.

Warning from Anselin and Li regarding this inference strategy:

    Note that this measure only addresses the so-called compositional or a-spatial characteristics of the pattern (following the terminology of Boots 2003, 2006). In other words, it focuses on how many of the neighbors have a value xj = 1 , but ignores where these nonzero values occur among the neighbor sites (what Boots 2006, refers to as the configuration). In Boots (2006), several approaches are proposed to analyze patterns on regular lattice structures. However, the proposed methods do not generalize to irregular spatial layouts.

2. Conditional permutation test

The conditional permutation test approach is used extensively throughout pysal and should be readily implementable. 

    A conditional permutation test as proposed in Anselin (1995) to compute a pseudo p value for the LISA statistics can be constructed in the usual way. The general principle, for those locations i where xi = 1 , is to carry out a series of random permutations of the remaining observations, while counting the times the number of neighbors with xj = 1 equals or exceeds qi , the observed value of the join counts. In practice, this is implemented by taking ki (the number of neighbors for i) draws without replacement from a set of N − 1 observations with K − 1 values of 1 for those observations where xi = 1 . A pseudo p value can be computed as (v + 1)∕(r + 1) , where v is the number of times the neighbors have qi or more values equal 1, and r is the number of permutations. The standard caveats apply (e.g., sensitivity to the number of permutations, varying results depending on the random number sequence, multiple comparisons, etc.). It should be noted that, as a one-sided test, the conditional permutation approach includes instances as rejecting the null of spatial randomness where there are more than qi neighbors with xj = 1 in the computation of the pseudo p value.



# Bivariate local join count statistics

Anselin and Li distinguish two cases of the bivariate join count statistic:

1. Without in-situ co-location (i.e. co-location in i, but not in neighbors). This is a situation where $x_i$ and $z_i$ do NOT take on the same value at $i$ or $j$.

This would be the example where $x_i=1$ for location $i$ and the other variable $z_i=0$. This may be useful when x and z correspond 'to two different values of a single categorical variable [that has been split across many columns]'.

Formally the bivariate local join count (BJC) statistic:

$$ BJC_i = x_{i}(1-z_{i}) \sum_j w_{ij} z_j (1-x_j)$$

To reiterate, the above situation only works when $z_i=0$ and $x_i = 1$. While x and z can be reversed, the statistic is not symmetric, so the results may be different whether x or z is the focus.

Another special part of this is when $x_i \neq z_i \forall i$, the statistic can be simplified to:

$$ BJC_i = x_i \sum_j w_{ij} z_j $$

**This is important and should probably be done with an `if` check at the beginning ish of the function.** 

Inference is gained from the one-sided conditional permutation test:

This is implemented by carrying out a series of ki draws for each location i where xi = 1 and zi = 0 . The draws are without replacement from N − 1 data tuples (xj, zj) of which Q observations have z = 1 (since zi = 0 ) and P − 1 observations have x = 1 .11 In practice, we only need to draw the zj , since the matching xj are zero by construction. The number of times the resulting local join count statistic from Eq. 3 equals or exceeds the observed value yields a pseudo p value.


2. with in-situ co-location (i.e. co-locaiton in i and in neighbors). This is when the interest in co-located events are surrounded by other co-located events. Also known as a co-location cluster(CLC)

This requires $x_i=z_i=1$ as well as $x_j=z_j=1$ for the neighbors. Formally:

$$ CLC_i = x_i z_i \sum_j w_{ij} x_j z_j $$

This effectively becomes the 1 times the sum of events that are also both 1 in neighboring units.

Inference is gained from:

    A conditional permutation approach can be constructed for those locations with xi = zi = 1 . We draw ki pairs of observations (xj, zj) from the set of N − 1 (this contains P − 1 observations with xj = 1 and Q − 1 observations with zj = 1 ). In a onesided test, we again count the number of times the statistic in Eq. 4 equals or exceeds the observed join count value at i.

## Extending the CLC to the multivariate context

First and foremost, the 'curse of dimensionality'. Results become less meaningful as the number of variables increase beacuse the coincidences are increasingly rare, and thus always indicated as significant. (**Perhaps opportunity for bonferonni or some other type of correction? or come up with one?**)

Focusing on the situation of the CLC, we consider $m$ variables at location $i$, i.e. $x_{hi}$ for $h = 1, ... , m$, with $\Pi^m_{h=1} x_{hi} = 1$ (i.e. conditional upon co-location of these variables at i). The corresponding co-locaiton cluster statistic this then:

$$ CLC_i = \Pi^m_{h=1} x_{hi} \sum_j w_{ij} \Pi^m_{h=1} x_{hj} $$

Inference:

    The implementation of a conditional permutation strategy follows as a direct generalization of the bivariate co-location cluster. However, as pointed out, for a large number of variables, such co-locations become less and less likely, and a different conceptual framework may be more appropriate.



# Concluding thoughts

Having reviewed the content of the paper, I think the next step is to look at the `join_counts` function of esda and see how the global statistic is implement. 

I forsee some challenges with inference and extending the statistic to the multivariate context. 